Note: This notebook makes use of a polygon of england when determining where a brownfield site is located. This polygon can be downloaded here: https://cartographyvectors.com/map/1321-england-uk, by selecting 'Choose Map Format' and then 'Well-Known-Text (WKT)'. The polygon is then stored in the same directory as this notebook.

In [1]:
england_poly_json = "englandjson.wkt"
england_json = open(england_poly_json, "r").read()

In [2]:
import urllib
import pandas as pd
import shapely.wkt
from shapely.geometry import shape
from shapely.errors import WKTReadingError
from shapely.geometry import Point
from shapely.ops import transform
import json
from urllib.request import urlopen

/tmp/ipykernel_8702/1460913449.py:5: DeprecationWarning: WKTReadingError is deprecated and will be removed in a future version. Use ShapelyError instead (functions previously raising {name} will now raise a ShapelyError instead).
  from shapely.errors import WKTReadingError


In [3]:
def get_all_organisations():
    params = urllib.parse.urlencode({
        "sql": f"""
        select organisation, name, entity as organisation_entity, statistical_geography
        from organisation
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/digital-land.csv?{params}"
    df = pd.read_csv(url)
    return df

def get_la_district_json(lpa_ref):
    params = urllib.parse.urlencode({
        "sql": f"""
        select json
        from entity
        where reference = '{lpa_ref}'
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/entity.csv?{params}"
    df = pd.read_csv(url)
    try:
        return df.loc[0,"json"]
    except KeyError:
        return None

def get_brownfield_sites_for_organisation(organisation_entity_number):
    params = urllib.parse.urlencode({
        "sql": f"""
        select json, point, reference, organisation_entity
        from entity
        where organisation_entity = '{organisation_entity_number}'
        and end_date = ''
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/brownfield-land.csv?{params}"
    df = pd.read_csv(url)
    return df

In [4]:
def get_LPA_multipolygon(reference):
    params = urllib.parse.urlencode({
        "sql": f"""
        select geometry, point
        from entity
        where reference = '{reference}'
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/entity.csv?{params}"
    df = pd.read_csv(url)
    try:
        return df.loc[0,"geometry"]
    except KeyError:
        return None

def get_site_point(collection_name, entity_number):
    params = urllib.parse.urlencode({
        "sql": f"""
        select point
        from entity
        where entity = '{entity_number}'
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/{collection_name}.csv?{params}"
    df = pd.read_csv(url)
    return df.loc[0,"point"]

def parse_wkt(value):
    try:
        geometry = shapely.wkt.loads(value)
    except WKTReadingError:
        try:
            geometry = shapely.wkt.loads(shape(json.loads(value)).wkt)
            return geometry, "invalid type geojson"
        except Exception:
            return None, "invalid WKT"
    return geometry, None


def make_point(point):
    if point.geom_type == "Point":
        return Point(point)
    else:
        print("Not a point")

In [5]:
def compute_true_location(pt, england_poly):
    if (not pt.within(england_poly[0])):
        return "Not in England"
    url = f"https://api.postcodes.io/postcodes?lon={pt.x}&lat={pt.y}"
    response = urlopen(url)
    data = json.loads(response.read())
    try:
        location = data["result"][0]["admin_district"]
        return location
    except Exception:
        return "None found"

def compute_error_in_coordinates(pt, area):
    amended_pt = transform(lambda x, y: (-x, y), pt)
    if (amended_pt.within(area)):
        return "Wrong sign on x co-ordinate"
    return "Unknown error"

In [6]:
include_null_coordinate_data = False
collection="brownfield_land"
england_poly = parse_wkt(england_json)

df_lpa = get_all_organisations()
df_brownfield_sites_outside_lpa = pd.DataFrame(columns=["Site_Reference", "Organisation", "Organisation_Name", "Point", "Maps_Link", "Admin_District", "Distance (Arbitrary Unit)", "Error"])
for lpa in df_lpa.itertuples():
    df_brownfield_sites = get_brownfield_sites_for_organisation(lpa.organisation_entity)
    df_brownfield_sites = df_brownfield_sites.merge(df_lpa, left_on="organisation_entity", right_on="organisation_entity")
    if ("local-authority-eng" in lpa.organisation):
        multipol = get_LPA_multipolygon(lpa.statistical_geography)
        if multipol is not None:
            area, issue = parse_wkt(multipol)
            for site in df_brownfield_sites.itertuples():
                if (pd.isnull(site.point) == False):
                    pt = shapely.wkt.loads(site.point)
                    if (pt.within(area) == False):                       
                        admin_district = compute_true_location(pt, england_poly)
                        distance = area.boundary.distance(pt)
                        error = compute_error_in_coordinates(pt, area)
                        google_maps_link = f"https://maps.google.com/?q={pt.y},{pt.x}"
                        pt_outside_boundary_row = {"Site_Reference": site.reference, "Organisation": lpa.organisation, "Organisation_Name": lpa.name, "Point": site.point, "Maps_Link": google_maps_link, "Admin_District": admin_district, "Distance (Arbitrary Unit)": distance, "Error": error}
                        df_brownfield_sites_outside_lpa = pd.concat([df_brownfield_sites_outside_lpa, pd.DataFrame([pt_outside_boundary_row])] , ignore_index=True)
                elif (include_null_coordinate_data):
                    pt_no_coord_row = {"Site_Reference": site.reference, "Organisation": lpa.organisation, "Organisation_Name": lpa.name, "Point": "No coordinate data"}
                    df_brownfield_sites_outside_lpa = pd.concat([df_brownfield_sites_outside_lpa, pd.DataFrame([pt_no_coord_row])] , ignore_index=True)      

In [7]:
# Sort by distance:
df_brownfield_sites_outside_lpa_sorted = df_brownfield_sites_outside_lpa.sort_values(by=["Distance (Arbitrary Unit)"], ascending=False)

# Sort by organisation count:
# df_brownfield_sites_outside_lpa_counts = df_brownfield_sites_outside_lpa.organisation.value_counts()
# df_brownfield_sites_outside_lpa_sorted = df_brownfield_sites_outside_lpa.set_index("organisation").loc[df_brownfield_sites_outside_lpa_counts.index].reset_index()
print("CSV output has been saved under 'brownfield_sites_outside_lpa.csv'")
df_brownfield_sites_outside_lpa_sorted = df_brownfield_sites_outside_lpa_sorted.reset_index(drop=True)
df_brownfield_sites_outside_lpa_sorted.to_csv("brownfield_sites_outside_lpa.csv")
display(df_brownfield_sites_outside_lpa_sorted)

CSV output has been saved under 'brownfield_sites_outside_lpa.csv'


,Site_Reference,Organisation,Organisation_Name,Point,Maps_Link,Admin_District,Distance (Arbitrary Unit),Error
0,MDCBLR009,local-authority-eng:MAL,Maldon District Council,POINT(-6.944827 51.657811),"https://maps.google.com/?q=51.657811,-6.944827",Not in England,7.534320,Unknown error
1,SHLAAHOE14,local-authority-eng:WOI,Woking Borough Council,POINT(-5.743 54.354),"https://maps.google.com/?q=54.354,-5.743",Not in England,5.934111,Unknown error
2,SHLAAHOE13,local-authority-eng:WOI,Woking Borough Council,POINT(-5.743 54.354),"https://maps.google.com/?q=54.354,-5.743",Not in England,5.934111,Unknown error
3,SHLAAHOE12,local-authority-eng:WOI,Woking Borough Council,POINT(-5.74 54.351),"https://maps.google.com/?q=54.351,-5.74",Not in England,5.929997,Unknown error
4,SHLAAHOE006,local-authority-eng:WOI,Woking Borough Council,POINT(-5.74 54.351),"https://maps.google.com/?q=54.351,-5.74",Not in England,5.929997,Unknown error
...,...,...,...,...,...,...,...,...
641,BR8/07/0275,local-authority-eng:CHC,Christchurch Borough Council,POINT(-1.792815 50.736696),"https://maps.google.com/?q=50.736696,-1.792815",None found,0.000021,Unknown error
642,BLR050,local-authority-eng:LBH,London Borough of Lambeth,POINT(-0.123465 51.414672),"https://maps.google.com/?q=51.414672,-0.123465",Croydon,0.000021,Unknown error
643,APP/13/00317,local-authority-eng:HAA,Havant Borough Council,POINT(-0.96651 50.811252),"https://maps.google.com/?q=50.811252,-0.96651",Havant,0.000016,Unknown error
644,WP/WESW/003,local-authority-eng:WEY,Weymouth and Portland Borough Council,POINT(-2.482258 50.612186),"https://maps.google.com/?q=50.612186,-2.482258",Dorset,0.000011,Unknown error
